In [1]:
import pandas as pd
import numpy as np


In [2]:
properties_list = ['awareness',
       'change_of_location', 'change_of_state', 'changes_possession',
       'created', 'destroyed', 'existed_after', 'existed_before',
       'existed_during', 'exists_as_physical', 'instigation',
       'location_of_event', 'makes_physical_contact', 'manipulated_by_another',
       'predicate_changed_argument', 'sentient', 'stationary', 'volition']


In [6]:
headers = pd.read_csv("decomp_ud_modified_sentences.csv", header=None, nrows=2,
                      index_col=0, keep_default_na=False, na_filter=False).values.tolist()
roles_wide = pd.read_csv("decomp_ud_modified_sentences.csv", header=[0, 1], index_col=0)
roles_wide.columns = pd.MultiIndex.from_arrays(headers)

print(roles_wide.shape)
roles_wide.head()


(7499, 48)


,Sentence.ID,Pred.Lemma,Arg.Phrase,Pred.Token,Arg.Tokens.Begin,Arg.Tokens.End,Gram.Func,Split,Sent.Grammatical,Annotator.ID,...,tree,sentence,spacy_parse,Arg.Tree,Arg.treepos,Predicate,Arg.Stripped,Arg.Parse,Roleset,Arg
,,,,,,,,,,,...,,,,,,,,,,
0,en-ud-dev.conllu 1,come,this story,3,4,5,nsubj,dev,5.0,7,...,(SINV\n (PP-1 (IN From) (NP (DT the) (NNP AP)...,From the AP comes this story :,"{'text': 'From the AP comes this story : ', 's...",(NP-SBJ (DT this) (NN story)),"(2,)",comes,this story,"{'text': 'this story ', 'sents': [{'start': 0,...",come.03,1
1,en-ud-dev.conllu 1,come,this story,3,4,5,nsubj,dev,5.0,20,...,(SINV\n (PP-1 (IN From) (NP (DT the) (NNP AP)...,From the AP comes this story :,"{'text': 'From the AP comes this story : ', 's...",(NP-SBJ (DT this) (NN story)),"(2,)",comes,this story,"{'text': 'this story ', 'sents': [{'start': 0,...",come.03,1
2,en-ud-dev.conllu 100,cry,America,1,0,0,nsubj,dev,2.0,46,...,(S\n (S\n (NP-SBJ+NNP America)\n (VP (V...,"America cried wolf in Iraq , and what 's scary...","{'text': ""America cried wolf in Iraq, and what...",(NP-SBJ+NNP America),"(0, 0)",cried,America,"{'text': 'America ', 'sents': [{'start': 0, 'e...",cry.01,0
3,en-ud-dev.conllu 100,cry,America,1,0,0,nsubj,dev,5.0,15,...,(S\n (S\n (NP-SBJ+NNP America)\n (VP (V...,"America cried wolf in Iraq , and what 's scary...","{'text': ""America cried wolf in Iraq, and what...",(NP-SBJ+NNP America),"(0, 0)",cried,America,"{'text': 'America ', 'sents': [{'start': 0, 'e...",cry.01,0
4,en-ud-dev.conllu 100,cry,wolf,1,2,2,dobj,dev,5.0,26,...,(S\n (S\n (NP-SBJ+NNP America)\n (VP (V...,"America cried wolf in Iraq , and what 's scary...","{'text': ""America cried wolf in Iraq, and what...",(NP+NN wolf),"(0, 1, 1)",cried,wolf,"{'text': 'wolf ', 'sents': [{'start': 0, 'end'...",cry.01,1


In [9]:
roles_wide["arg_idx"] = roles_wide.apply(
    lambda x: (x[("Arg.Tokens.Begin", "")], x[("Arg.Tokens.End", "")] + 1),
    axis=1
    )
# verb always is a single token
roles_wide["verb_idx"] = roles_wide["Pred.Token"].apply(
    lambda x: (x, x+1)
)

roles_wide


,Sentence.ID,Pred.Lemma,Arg.Phrase,Pred.Token,Arg.Tokens.Begin,Arg.Tokens.End,Gram.Func,Split,Sent.Grammatical,Annotator.ID,...,spacy_parse,Arg.Tree,Arg.treepos,Predicate,Arg.Stripped,Arg.Parse,Roleset,Arg,arg_idx,verb_idx
,,,,,,,,,,,...,,,,,,,,,,
0,en-ud-dev.conllu 1,come,this story,3,4,5,nsubj,dev,5.0,7,...,"{'text': 'From the AP comes this story : ', 's...",(NP-SBJ (DT this) (NN story)),"(2,)",comes,this story,"{'text': 'this story ', 'sents': [{'start': 0,...",come.03,1,"(4, 6)","(3, 4)"
1,en-ud-dev.conllu 1,come,this story,3,4,5,nsubj,dev,5.0,20,...,"{'text': 'From the AP comes this story : ', 's...",(NP-SBJ (DT this) (NN story)),"(2,)",comes,this story,"{'text': 'this story ', 'sents': [{'start': 0,...",come.03,1,"(4, 6)","(3, 4)"
2,en-ud-dev.conllu 100,cry,America,1,0,0,nsubj,dev,2.0,46,...,"{'text': ""America cried wolf in Iraq, and what...",(NP-SBJ+NNP America),"(0, 0)",cried,America,"{'text': 'America ', 'sents': [{'start': 0, 'e...",cry.01,0,"(0, 1)","(1, 2)"
3,en-ud-dev.conllu 100,cry,America,1,0,0,nsubj,dev,5.0,15,...,"{'text': ""America cried wolf in Iraq, and what...",(NP-SBJ+NNP America),"(0, 0)",cried,America,"{'text': 'America ', 'sents': [{'start': 0, 'e...",cry.01,0,"(0, 1)","(1, 2)"
4,en-ud-dev.conllu 100,cry,wolf,1,2,2,dobj,dev,5.0,26,...,"{'text': ""America cried wolf in Iraq, and what...",(NP+NN wolf),"(0, 1, 1)",cried,wolf,"{'text': 'wolf ', 'sents': [{'start': 0, 'end'...",cry.01,1,"(2, 3)","(1, 2)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7494,en-ud-train.conllu 9973,have,They,1,0,0,nsubj,train,5.0,26,...,"{'text': 'They have a credit card minimum. ', ...",(NP-SBJ+PRP They),"(0,)",have,They,"{'text': 'They ', 'sents': [{'start': 0, 'end'...",have.03,0,"(0, 1)","(1, 2)"
7495,en-ud-train.conllu 9995,phone,I,1,0,0,nsubj,train,2.0,7,...,{'text': 'I phoned this company for advice on ...,(NP-SBJ+PRP I),"(0, 0)",phoned,I,"{'text': 'I ', 'sents': [{'start': 0, 'end': 1...",phone.01,0,"(0, 1)","(1, 2)"
7496,en-ud-train.conllu 9995,phone,I,1,0,0,nsubj,train,5.0,81,...,{'text': 'I phoned this company for advice on ...,(NP-SBJ+PRP I),"(0, 0)",phoned,I,"{'text': 'I ', 'sents': [{'start': 0, 'end': 1...",phone.01,0,"(0, 1)","(1, 2)"


In [10]:
new_index = [x[0] for x in roles_wide.columns if x[1] == '']
active_responses_df = roles_wide["Response"]
properties_list = active_responses_df.columns
print(properties_list)

active_responses_df[new_index] = roles_wide[new_index]
active_responses_df[properties_list] = active_responses_df[properties_list].to_numpy().astype(int)
active_responses_df["structure"] = "active_full"
active_responses_df = active_responses_df.set_index(new_index
).reset_index().drop(columns=["Pred.Token", "Arg.Tokens.Begin", "Arg.Tokens.End"])

active_responses_df.head()


Index(['awareness', 'change_of_location', 'change_of_possession',
       'change_of_state', 'change_of_state_continuous', 'existed_after',
       'existed_before', 'existed_during', 'instigation', 'partitive',
       'sentient', 'volition', 'was_for_benefit', 'was_used'],
      dtype='object')


/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_41791/570044630.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_responses_df[new_index] = roles_wide[new_index]
/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_41791/570044630.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_responses_df[new_index] = roles_wide[new_index]
/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_41791/570044630.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

,Sentence.ID,Pred.Lemma,Arg.Phrase,Gram.Func,Split,Sent.Grammatical,Annotator.ID,tree,sentence,spacy_parse,...,existed_after,existed_before,existed_during,instigation,partitive,sentient,volition,was_for_benefit,was_used,structure
0,en-ud-dev.conllu 1,come,this story,nsubj,dev,5.0,7,(SINV\n (PP-1 (IN From) (NP (DT the) (NNP AP)...,From the AP comes this story :,"{'text': 'From the AP comes this story : ', 's...",...,4,2,4,2,2,2,1,2,1,active_full
1,en-ud-dev.conllu 1,come,this story,nsubj,dev,5.0,20,(SINV\n (PP-1 (IN From) (NP (DT the) (NNP AP)...,From the AP comes this story :,"{'text': 'From the AP comes this story : ', 's...",...,5,5,5,5,2,1,3,3,5,active_full
2,en-ud-dev.conllu 100,cry,America,nsubj,dev,2.0,46,(S\n (S\n (NP-SBJ+NNP America)\n (VP (V...,"America cried wolf in Iraq , and what 's scary...","{'text': ""America cried wolf in Iraq, and what...",...,5,5,5,5,5,5,5,5,5,active_full
3,en-ud-dev.conllu 100,cry,America,nsubj,dev,5.0,15,(S\n (S\n (NP-SBJ+NNP America)\n (VP (V...,"America cried wolf in Iraq , and what 's scary...","{'text': ""America cried wolf in Iraq, and what...",...,5,5,5,5,5,5,5,3,5,active_full
4,en-ud-dev.conllu 100,cry,wolf,dobj,dev,5.0,26,(S\n (S\n (NP-SBJ+NNP America)\n (VP (V...,"America cried wolf in Iraq , and what 's scary...","{'text': ""America cried wolf in Iraq, and what...",...,3,3,3,3,3,1,3,3,3,active_full


In [11]:
active_responses_df.columns


Index(['Sentence.ID', 'Pred.Lemma', 'Arg.Phrase', 'Gram.Func', 'Split',
       'Sent.Grammatical', 'Annotator.ID', 'tree', 'sentence', 'spacy_parse',
       'Arg.Tree', 'Arg.treepos', 'Predicate', 'Arg.Stripped', 'Arg.Parse',
       'Roleset', 'Arg', 'arg_idx', 'verb_idx', 'awareness',
       'change_of_location', 'change_of_possession', 'change_of_state',
       'change_of_state_continuous', 'existed_after', 'existed_before',
       'existed_during', 'instigation', 'partitive', 'sentient', 'volition',
       'was_for_benefit', 'was_used', 'structure'],
      dtype='object')

In [13]:
passive_roles_wide = pd.read_csv("decomp_ud_passive_responses.csv").drop(columns=["Unnamed: 0"])

# flip the gram of the active sentence to passive sentence
passive_roles_wide["Gram.Func"] = passive_roles_wide["Gram.Func"].apply(
    lambda x: "nsubj" if x == "dobj" else "dobj")
passive_roles_wide["structure"] = "passive_full"
passive_roles_wide['arg_idx'] = passive_roles_wide.apply(
    lambda x: x[f"passive {x['Gram.Func']} idx"], axis=1
)
passive_roles_wide['verb_idx'] = passive_roles_wide["passive verb idx"]

for col in passive_roles_wide.columns:
    if "active" in col:
        passive_roles_wide.drop(columns=col, inplace=True)

passive_roles_wide["Arg.Phrase"] = passive_roles_wide.apply(
    lambda x: x["passive dobj"] if x["Gram.Func"] == "dobj" else x["passive nsubj"], axis=1
)

passive_roles_wide = passive_roles_wide.rename(
    columns={"passive sentence":"sentence", "passive verb":"Predicate"}
    )

for col in passive_roles_wide.columns:
    if "passive" in col:
        passive_roles_wide.drop(columns=col, inplace=True)

print(passive_roles_wide.shape)
passive_roles_wide.head()


(3910, 29)


,Sentence.ID,Pred.Lemma,sentence,Predicate,awareness,change_of_location,change_of_possession,change_of_state,change_of_state_continuous,existed_after,...,Annotator.ID,Arg.Phrase,Arg.Stripped,Gram.Func,Split,Roleset,Arg,structure,arg_idx,verb_idx
0,en-ud-dev.conllu 1002,call,"The refugee issue , was called by abbas , a re...",was called by,5.0,2.0,1.0,3.0,1.0,5.0,...,26,"abbas , a refugee himself from what is today t...",Abbas,dobj,dev,call.01,1,passive_full,"(7, 22)","(4, 7)"
1,en-ud-dev.conllu 1002,call,"The refugee issue , was called by abbas , a re...",was called by,4.0,3.0,1.0,4.0,4.0,5.0,...,76,"abbas , a refugee himself from what is today t...",Abbas,dobj,dev,call.01,1,passive_full,"(7, 22)","(4, 7)"
2,en-ud-dev.conllu 1002,call,"The refugee issue , was called by abbas , a re...",was called by,1.0,1.0,1.0,3.0,1.0,5.0,...,26,The refugee issue,the refugee issue,nsubj,dev,call.01,NaN,passive_full,"(0, 3)","(4, 7)"
3,en-ud-dev.conllu 1002,call,"The refugee issue , was called by abbas , a re...",was called by,5.0,4.0,1.0,5.0,4.0,5.0,...,93,The refugee issue,the refugee issue,nsubj,dev,call.01,NaN,passive_full,"(0, 3)","(4, 7)"
4,en-ud-dev.conllu 1013,resume,"In response to the rockets , its policy of tar...",was resumed by,5.0,1.0,1.0,5.0,5.0,5.0,...,43,israel,Israel,dobj,dev,resume.01,NaN,passive_full,"(18, 19)","(15, 18)"


In [14]:
roles_combined_v2 = pd.concat([active_responses_df, passive_roles_wide]).reset_index(drop=True).drop(
    columns=["tree", "spacy_parse", "Arg.Tree", "Arg.treepos", "Arg.Parse"]
)

roles_combined_v2.to_csv("active_passive_decompV2.csv")

print(roles_combined_v2.columns)
print(roles_combined_v2.shape)
roles_combined_v2.head()


Index(['Sentence.ID', 'Pred.Lemma', 'Arg.Phrase', 'Gram.Func', 'Split',
       'Sent.Grammatical', 'Annotator.ID', 'sentence', 'Predicate',
       'Arg.Stripped', 'Roleset', 'Arg', 'arg_idx', 'verb_idx', 'awareness',
       'change_of_location', 'change_of_possession', 'change_of_state',
       'change_of_state_continuous', 'existed_after', 'existed_before',
       'existed_during', 'instigation', 'partitive', 'sentient', 'volition',
       'was_for_benefit', 'was_used', 'structure', 'Arg.Tokens.Begin'],
      dtype='object')
(11409, 30)


,Sentence.ID,Pred.Lemma,Arg.Phrase,Gram.Func,Split,Sent.Grammatical,Annotator.ID,sentence,Predicate,Arg.Stripped,...,existed_before,existed_during,instigation,partitive,sentient,volition,was_for_benefit,was_used,structure,Arg.Tokens.Begin
0,en-ud-dev.conllu 1,come,this story,nsubj,dev,5.0,7,From the AP comes this story :,comes,this story,...,2.0,4.0,2.0,2.0,2.0,1.0,2.0,1.0,active_full,NaN
1,en-ud-dev.conllu 1,come,this story,nsubj,dev,5.0,20,From the AP comes this story :,comes,this story,...,5.0,5.0,5.0,2.0,1.0,3.0,3.0,5.0,active_full,NaN
2,en-ud-dev.conllu 100,cry,America,nsubj,dev,2.0,46,"America cried wolf in Iraq , and what 's scary...",cried,America,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,active_full,NaN
3,en-ud-dev.conllu 100,cry,America,nsubj,dev,5.0,15,"America cried wolf in Iraq , and what 's scary...",cried,America,...,5.0,5.0,5.0,5.0,5.0,5.0,3.0,5.0,active_full,NaN
4,en-ud-dev.conllu 100,cry,wolf,dobj,dev,5.0,26,"America cried wolf in Iraq , and what 's scary...",cried,wolf,...,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0,active_full,NaN


In [15]:
roles_combined_v2["decomp_version"] = "V2"
roles_combined_v2 = roles_combined_v2.rename(columns={
    "sentence": "Sentence"
}).drop(columns=["Sent.Grammatical", "was_used",
                 "change_of_state_continuous", "was_for_benefit", "partitive",
                 "Annotator.ID"])

# set nsubjpass as "passive" structure
roles_combined_v2["modified_sentence"] = roles_combined_v2.apply(
    lambda x: 1 if x["structure"] == "passive_full" else 0,
    axis=1
)
roles_combined_v2["structure"] = roles_combined_v2.apply(
    lambda x: "passive_full" if x["Gram.Func"] == "nsubjpass" else x["structure"],
    axis=1
)


roles_combined_v2["Gram.Func"] = roles_combined_v2["Gram.Func"].apply(
    lambda x: "nsubj" if x == "nsubjpass" else x
)


In [16]:
gram_map = {"obj": "dobj", "subj": "nsubj", "other": "iobj"}

roles_combined_v1 = pd.read_csv("../dataV1/active_passive_decompV1.csv").drop(columns=["Unnamed: 0"])
roles_combined_v1["decomp_version"] = "V1"
roles_combined_v1["Pred.Lemma"] = roles_combined_v1["Roleset"].apply(lambda x: x.split(".")[0])
roles_combined_v1["modified_sentence"] = roles_combined_v1.apply(
    lambda x: 1 if x["structure"] == "passive_full" else 0,
    axis=1
)
roles_combined_v1["Gram.Func"] = roles_combined_v1["Gram.Func"].apply(
    lambda x: gram_map[x]
)
roles_combined_v1.columns
roles_combined_v1 = roles_combined_v1.drop(columns=[
    "stationary", "exists_as_physical", "location_of_event",
    "makes_physical_contact", "predicate_changed_argument", "created", "destroyed", "manipulated_by_another"
]).rename(columns={"changes_possession":"change_of_possession"})

roles_combined_v1.head()


,Sentence.ID,Roleset,Gram.Func,Sentence,Predicate,Split,Arg.Phrase,awareness,change_of_location,change_of_state,...,sentient,volition,Arg.Stripped,Arg,arg_idx,verb_idx,structure,decomp_version,Pred.Lemma,modified_sentence
0,0003_21,impose.01,nsubj,"In July , a gradual ban was imposed by the Env...",was imposed by,test,a gradual ban,1.0,1.0,3.0,...,1.0,1.0,a ban,1,"(3, 6)","(6, 9)",passive_full,V1,impose,1
1,0003_21,impose.01,dobj,"In July , a gradual ban was imposed by the Env...",was imposed by,train,the Environmental Protection Agency,5.0,1.0,3.0,...,1.0,5.0,the Environmental Protection Agency,0,"(9, 13)","(6, 9)",passive_full,V1,impose,1
2,0003_29,have.03,nsubj,No bearing on our work force today is had by it .,is had by,train,No bearing on our work force today,1.0,1.0,1.0,...,1.0,1.0,no bearing,1,"(0, 7)","(7, 10)",passive_full,V1,have,1
3,0003_29,have.03,dobj,No bearing on our work force today is had by it .,is had by,test,it,3.0,3.0,3.0,...,2.0,3.0,It,0,"(10, 11)","(7, 10)",passive_full,V1,have,1
4,0003_9,lead.02,nsubj,A team of researchers from the National Cancer...,was led by,train,A team of researchers from the National Cancer...,5.0,3.0,3.0,...,1.0,5.0,a team,1,"(0, 19)","(19, 22)",passive_full,V1,lead,1


In [17]:
combined_versions = pd.concat([roles_combined_v1, roles_combined_v2]).reset_index(drop=True)
print(combined_versions.shape)

properties_list = ['awareness', 'change_of_location', 'change_of_state',
       'change_of_possession', 'existed_after', 'existed_before',
       'existed_during', 'instigation', 'sentient', 'volition']
combined_versions['combined_labels'] = combined_versions[properties_list].values.astype(float).tolist()
combined_versions.drop(columns=properties_list, inplace=True)
combined_versions.head()


(24906, 26)


,Sentence.ID,Roleset,Gram.Func,Sentence,Predicate,Split,Arg.Phrase,Arg.Stripped,Arg,arg_idx,verb_idx,structure,decomp_version,Pred.Lemma,modified_sentence,Arg.Tokens.Begin,combined_labels
0,0003_21,impose.01,nsubj,"In July , a gradual ban was imposed by the Env...",was imposed by,test,a gradual ban,a ban,1,"(3, 6)","(6, 9)",passive_full,V1,impose,1,NaN,"[1.0, 1.0, 3.0, 1.0, 5.0, 1.0, 5.0, 1.0, 1.0, ..."
1,0003_21,impose.01,dobj,"In July , a gradual ban was imposed by the Env...",was imposed by,train,the Environmental Protection Agency,the Environmental Protection Agency,0,"(9, 13)","(6, 9)",passive_full,V1,impose,1,NaN,"[5.0, 1.0, 3.0, 1.0, 5.0, 5.0, 5.0, 5.0, 1.0, ..."
2,0003_29,have.03,nsubj,No bearing on our work force today is had by it .,is had by,train,No bearing on our work force today,no bearing,1,"(0, 7)","(7, 10)",passive_full,V1,have,1,NaN,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
3,0003_29,have.03,dobj,No bearing on our work force today is had by it .,is had by,test,it,It,0,"(10, 11)","(7, 10)",passive_full,V1,have,1,NaN,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 5.0, 3.0, 2.0, ..."
4,0003_9,lead.02,nsubj,A team of researchers from the National Cancer...,was led by,train,A team of researchers from the National Cancer...,a team,1,"(0, 19)","(19, 22)",passive_full,V1,lead,1,NaN,"[5.0, 3.0, 3.0, 1.0, 3.0, 3.0, 5.0, 3.0, 1.0, ..."


In [18]:
combined_versions.columns


Index(['Sentence.ID', 'Roleset', 'Gram.Func', 'Sentence', 'Predicate', 'Split',
       'Arg.Phrase', 'Arg.Stripped', 'Arg', 'arg_idx', 'verb_idx', 'structure',
       'decomp_version', 'Pred.Lemma', 'modified_sentence', 'Arg.Tokens.Begin',
       'combined_labels'],
      dtype='object')

### Verbnet Alignment

In [19]:
import json
import re
# verbnet 3.4 imported from https://uvi.colorado.edu/nlp_applications#download

uvi = json.loads(open("uvi_export.json").read())

pb = {}
for frame in uvi["PropBank"]:
    for predicate in frame["predicates"]:
        for roleset in predicate["rolesets"]:
            roleset_id = roleset["roleset_id"]

            pb[roleset_id] = {}
            args = roleset["roles"]
            for arg in args:
                n = arg["n"]
                if n is None or arg["vnroles"] is None:
                    continue
                
                pb[roleset_id][n] = []
                for vn_mapping in arg["vnroles"]:
                    try:
                        vn_id, vn_role = re.sub(r'([0-9|.|-]+)-([a-z|-]+)',
                                                r'\1__\2', vn_mapping).split("__")
                        pb[roleset_id][n].append((vn_id, vn_role))
                    except:
                        print(vn_mapping)


-result


In [20]:
def map_to_verbnet(roleset, arg_id):
    try:
        return pb[roleset][arg_id]
    except:
        return np.nan
    
combined_versions["VN_mappings"] = combined_versions.apply(
    lambda x: map_to_verbnet( x["Roleset"], x["Arg"]), axis=1)
combined_versions


,Sentence.ID,Roleset,Gram.Func,Sentence,Predicate,Split,Arg.Phrase,Arg.Stripped,Arg,arg_idx,verb_idx,structure,decomp_version,Pred.Lemma,modified_sentence,Arg.Tokens.Begin,combined_labels,VN_mappings
0,0003_21,impose.01,nsubj,"In July , a gradual ban was imposed by the Env...",was imposed by,test,a gradual ban,a ban,1,"(3, 6)","(6, 9)",passive_full,V1,impose,1,NaN,"[1.0, 1.0, 3.0, 1.0, 5.0, 1.0, 5.0, 1.0, 1.0, ...",NaN
1,0003_21,impose.01,dobj,"In July , a gradual ban was imposed by the Env...",was imposed by,train,the Environmental Protection Agency,the Environmental Protection Agency,0,"(9, 13)","(6, 9)",passive_full,V1,impose,1,NaN,"[5.0, 1.0, 3.0, 1.0, 5.0, 5.0, 5.0, 5.0, 1.0, ...",NaN
2,0003_29,have.03,nsubj,No bearing on our work force today is had by it .,is had by,train,No bearing on our work force today,no bearing,1,"(0, 7)","(7, 10)",passive_full,V1,have,1,NaN,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",NaN
3,0003_29,have.03,dobj,No bearing on our work force today is had by it .,is had by,test,it,It,0,"(10, 11)","(7, 10)",passive_full,V1,have,1,NaN,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 5.0, 3.0, 2.0, ...",NaN
4,0003_9,lead.02,nsubj,A team of researchers from the National Cancer...,was led by,train,A team of researchers from the National Cancer...,a team,1,"(0, 19)","(19, 22)",passive_full,V1,lead,1,NaN,"[5.0, 3.0, 3.0, 1.0, 3.0, 3.0, 5.0, 3.0, 1.0, ...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24901,en-ud-train.conllu 9973,have.03,dobj,A credit card minimum is had by them .,is had by,train,them,They,0,"(7, 8)","(4, 7)",passive_full,V2,have,1,0.0,"[5.0, 2.5, 3.5, 2.0, 5.0, 4.0, 5.0, 5.0, 5.0, ...","[(100.1, pivot), (39.4, agent)]"
24902,en-ud-train.conllu 9995,phone.01,dobj,This company was phoned by me for advice on ou...,was phoned by,train,me,I,0,"(5, 6)","(2, 5)",passive_full,V2,phone,1,0.0,"[5.0, 3.0, 2.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[(13.5.1, agent), (37.4.2-1, agent), (37.4.1, ..."
24903,en-ud-train.conllu 9995,phone.01,dobj,This company was phoned by me for advice on ou...,was phoned by,train,me,I,0,"(5, 6)","(2, 5)",passive_full,V2,phone,1,0.0,"[5.0, 2.0, 5.0, 2.0, 5.0, 5.0, 5.0, 5.0, 4.0, ...","[(13.5.1, agent), (37.4.2-1, agent), (37.4.1, ..."
24904,en-ud-train.conllu 9995,phone.01,nsubj,This company was phoned by me for advice on ou...,was phoned by,train,This company,this company,1,"(0, 2)","(2, 5)",passive_full,V2,phone,1,2.0,"[5.0, 2.0, 2.0, 2.0, 5.0, 5.0, 5.0, 1.0, 1.0, ...","[(37.4.2-1, recipient), (37.4.1, recipient)]"


In [21]:
combined_versions.to_csv("combined_V1_V2.csv")


In [22]:
combined_no_duplicates = combined_versions.drop(columns="VN_mappings")
combined_no_duplicates = combined_no_duplicates.drop_duplicates(
    subset=[x for x in combined_no_duplicates.columns if x != "combined_labels"]
)
print(combined_no_duplicates.shape)

for column in ["Gram.Func", "Split", "structure", "decomp_version", "modified_sentence", "Arg"]:
    print(combined_no_duplicates[column].value_counts())


(19246, 17)
Gram.Func
nsubj    10661
dobj      6605
iobj      1980
Name: count, dtype: int64
Split
train    15306
test      1983
dev       1957
Name: count, dtype: int64
structure
active_full     13231
passive_full     6015
Name: count, dtype: int64
decomp_version
V1    13497
V2     5749
Name: count, dtype: int64
modified_sentence
0    13470
1     5776
Name: count, dtype: int64
Arg
1    6017
0    5703
0    2417
1    2068
2    1366
4     236
2     177
3     160
R     121
J      43
P      23
5      15
S       5
C       4
3       4
D       3
U       3
4       2
Name: count, dtype: int64
